# Problematic

#### For sociologists, to study the impact of stress and high-pressure environments on one’s health. They are looking for factors in people’s lives that could correlate to certain diseases or conditions. : 

They don't need to know any information about the identity of individuals. However, gender, dob and localisaiton  is really important information : 

- The id ( by name could be dropped (BE CARREFUL BECAUSE THIS INFORMATION COULD BE LINKED WITH THE SECOND PART OF THE EXERCICE)) 
- The gender could be replaced by 0 or 1 ( mais en vrai ca change pas grand chose)
- The dob : we don't need the day and the month of birth ==> Ne pourrait-on pas modifier grâce à une gaussienne de moyenne 0 sd 1 les dates de naissance ? On gagne en anonymisation, on ne perd pas grand chose 
- Zip code, we do not need the entire zipcode, we can keep only the first to digit.

==> Etant donnél'étude demandé, le risque n'est pas les identifiant mais les pseusos identifiants.

#### For the US department of Health and Human services chairman, to decide where to build new hospitals, and which departments (radiology, neurology, pulmonology, oncology...) to have in these, as the US federal government has just allocated money to build *5 new hospitals across the country* : 

A priori, je ne vois pas pourquoi on aurait besoin d'information sur les individus, ce qui nous importe est uniquement les maladies, la localicaiton, et pourquoi pas l'age des gens.

- Gender ne sert plus à rien
- l'année de naissance suffit 
- les deux premiers digit du zip code sont suffisant (étant donné que l'on va faire que 5 hopitauyx across the country)
- L'éducation on s'en branle
- Emploie on s'en fou
- Nombre d'enfant on s'en fou 
- Marital status on s'en branle
- ancestry on s'en fou 
- Nombre de véhicule ( est ce que ca peut être intéressant étant donné que c'est des hopitaux? ==> Ambulance tout ca)
- communte-time on s'en branle 
- accommodation on s'en fou 
- Maladie, pourrait on déjà regrouper les différentes maladies selon le service utilise à soigner les tiches ? On s'en branle de savoir si c'est un cancer du foie ou de la peau, ce qui est intéressant c'est qu'il faut un service d'oncologie



# Packages and data imports

## Import packages

In [30]:
#conda install -c conda-forge sklearn-pandas
import pandas as pd 
import os
import datetime as dt
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn_pandas import DataFrameMapper
from sklearn.cluster import KMeans
import hashlib as h
from collections import Counter
import math


## Data import

In [31]:
data = pd.read_csv("../resources/dataset_HW3.csv")
states = pd.read_csv("../resources/states.csv")

In [32]:
data

,id,gender,dob,zipcode,education,employment,children,marital_status,ancestry,number_vehicles,commute_time,accommodation,disease
0,Amelia Morrow,female,12/1950,26904,High School,Retired,2,married,Africa,2,0.0,Own house,Alzheimer's disease
1,Lillian Cole,female,4/1940,78088,PhD/md,Retired,1,married,Africa,1,0.0,Rent flat,multiple sclerosis
2,Lillian Simpson,female,12/1966,51502,Bachelor,Employed,1,married,Asia,0,0.1,Rent flat,heart disease
3,Avery Richards,female,4/1943,54080,Bachelor,Retired,1,married,Africa,0,0.0,Rent room,endometriosis
4,Sophia Alvarado,female,10/1947,68785,High School,Retired,1,married,Europe,0,0.0,Rent flat,gastritis
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,Connor Pena,male,11/1997,71300,High School,Student,0,divorced,Europe,0,1.1,Rent house,gastritis
1996,Caleb Bowman,male,11/1999,49030,High School,Student,0,single,North America,0,0.1,Rent house,gastritis
1997,Ella Barber,female,6/1948,92883,Masters,Retired,5,married,Africa,1,0.0,Own house,skin cancer
1998,Gavin Wilson,male,4/1976,42200,High School,Employed,1,married,Asia,2,0.0,Own flat,diabetes


# ID anonymisation

In [33]:
def ID_anonymised_df(df, secret_key):
    new_id_vec = []
    for id in df['id']:
        id_salted = str(id.replace(' ', '')+secret_key)
        new_id_vec.append(h.sha3_256(id_salted.encode()).hexdigest())
    
    anonimsed_df = df.copy()
    anonimsed_df['id'] = new_id_vec

    return anonimsed_df


In [34]:
data_socio = ID_anonymised_df(data, 'arthurgeeraerd11751600')
data_hospi = ID_anonymised_df(data, 'simonadrienkinartlegendre')

# Transformations 

### Date of birth

In [35]:
# Projet hopitaux : on met les ages par dizaine d'année
for i in range(len(data_hospi["dob"])):

    # Modify DOB 
    s = 0 #np.random.normal(0, 2, 1)
    data_hospi.at[i,"dob"] = str(round(int(data_hospi["dob"][i][-4:]) +s, -1))[-2:]#[0])  # ARRONDI A LA DIZAINE


# Projet sociologique : on met les ages avec une erreur suivant une normale. ++> PERMET EN PLUS DE DIFFERENCIER par rapport à l'autre BDD.

for i in range(len(data_socio["dob"])):
    # Modify DOB 
    #s = np.random.normal(0, 2, 1)
    #data_socio.at[i,"dob"] = round(int(data_socio["dob"][i][-4:]) +s[0]) 
    data_socio.at[i,"dob"] = str(round(int(data_hospi["dob"][i][-4:]) +s, -1))[-2:]#[0])  # ARRONDI A LA DIZAINE


In [36]:
data_socio

,id,gender,dob,zipcode,education,employment,children,marital_status,ancestry,number_vehicles,commute_time,accommodation,disease
0,c47ed1e2401af117d792899356875808fc468009e00545...,female,50,26904,High School,Retired,2,married,Africa,2,0.0,Own house,Alzheimer's disease
1,9200e092cb89f3af990d24c9c6c4875f1b9a74c98657ba...,female,40,78088,PhD/md,Retired,1,married,Africa,1,0.0,Rent flat,multiple sclerosis
2,457e8ab0e940da0028da57f4e5dd4e71f5c7bda98c413b...,female,70,51502,Bachelor,Employed,1,married,Asia,0,0.1,Rent flat,heart disease
3,b10eefa100ba13cbd732cdb52bbbe6f0714fbc389cfbf4...,female,40,54080,Bachelor,Retired,1,married,Africa,0,0.0,Rent room,endometriosis
4,531e1ab7da669ece5b348da0d87c376e7285b816962aa3...,female,50,68785,High School,Retired,1,married,Europe,0,0.0,Rent flat,gastritis
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,9a7fe6c68979b2048d512322cd54c443475caec1eda79c...,male,0,71300,High School,Student,0,divorced,Europe,0,1.1,Rent house,gastritis
1996,c029c6b3fb9bb438c12197fc3696a161f0c93522b0b862...,male,0,49030,High School,Student,0,single,North America,0,0.1,Rent house,gastritis
1997,35926def0e1716ce7af3a8e9e459dac009b63ccb26677c...,female,50,92883,Masters,Retired,5,married,Africa,1,0.0,Own house,skin cancer
1998,8c98ee03166da806552dd3b0892d230c359317db757777...,male,80,42200,High School,Employed,1,married,Asia,2,0.0,Own flat,diabetes


## Mapping Variable catégorielles

Chaque fonction permet de gérer une variable catégorielle. On mets en entrée la valeur que l'on veut pour chaque catégorie possible, 
en retour, la database est modifiée avec les catégorie que l'on veut.

Exemple : 

change_education(data, "low", "low", "middle", "high", "high") va remplacer dans la dataBase Lessthan high school par low, high school par low, chachelor par moiddle, et master et phd par high

### functions

In [37]:
def change_education(data, less, high, bach, master, phd):

    mapping_education = {
        'Less than High School':less,
        'High School':high,
        'Bachelor':bach,
        'Masters':master,
        'PhD/md':phd,
    }
    changed_data = data.copy()
    changed_data["education"] = data.education.map(mapping_education)
    return changed_data


def change_gender(data, male, female):

    mapping_gender = {
        'male': male,
        'female': female
    }

    
    changed_data = data.copy()
    changed_data["gender"] = data.gender.map(mapping_gender)
    return changed_data


def change_employment(data, employed, retired, student, unemployed):
    mapping_employment = {
        'Employed' : employed,
        'Retired' : retired,
        'Student' : student,
        'Unemployed' : unemployed
    }

    changed_data = data.copy()
    changed_data["employment"] = data.employment.map(mapping_employment)
    return changed_data


def change_children(data, zero, un, deux, trois, quatre , cinq, six, sept, huit):
    mapping_children = {
        0 : zero,
        1 : un,
        2 : deux,
        3 : trois,
        4 : quatre,
        5 : cinq,
        6 : six,
        7 : sept,
        8 : huit
    }

    changed_data = data.copy()
    changed_data["children"] = data.children.map(mapping_children)
    return changed_data



def change_marital_status(data, divorced, married, single, widowed):
    mapping_marital_status = {
        "divorced" : divorced,
        "married" : married,
        "single" : single,
        "widowed" : widowed,

    }

    changed_data = data.copy()
    changed_data["marital_status"] = data.marital_status.map(mapping_marital_status)
    return changed_data


def change_ancestry(data, Africa, Asia, Europe, North_America, Oceania, South_America):
    mapping_ancestry = {
        "Africa" : Africa,
        "Asia" : Asia,
        "Europe" : Europe,
        "North America" : North_America,
        "Oceania" : Oceania,
        "South America" : South_America

    }

    changed_data = data.copy()
    changed_data["ancestry"] = data.ancestry.map(mapping_ancestry)
    return changed_data


def change_number_vehicles(data, zero, un, deux, trois):
    mapping_nmbr_vehicule = {
        0 : zero,
        1 : un,
        2 : deux,
        3 : trois,

    }

    changed_data = data.copy()
    changed_data["number_vehicles"] = data.number_vehicles.map(mapping_nmbr_vehicule)
    return changed_data


def change_accommodation(data, Own_flat, Own_house, Rent_flat, Rent_house, Rent_room):
    mapping_accommodation = {
        "Own flat" : Own_flat,
        "Own house" : Own_house,
        "Rent flat" : Rent_flat,
        "Rent house" : Rent_house,
        "Rent room" : Rent_room,
        
    }

    changed_data = data.copy()
    changed_data["accommodation"] = data.accommodation.map(mapping_accommodation)
    return changed_data

# for the sake of the first use case we gather all specific diseases under more general health department 

def change_disease(data, gastritis, heart_disease, hypertension, diabetes, skin_cancer, prostate_cancer, breast_cancer, schizophrenia, Alzheimer, multiple_sclerosis, endometriosis, kidney_disease, HIV_AIDS):
    mapping_disease = {
        'gastritis': gastritis ,
        'heart disease': heart_disease,
        "hypertension": hypertension,
        "diabetes": diabetes,
        "skin cancer": skin_cancer,
        "prostate cancer":  prostate_cancer,
        "breast cancer": breast_cancer, 
        "schizophrenia": schizophrenia,
        "Alzheimer's disease": Alzheimer, 
        "multiple sclerosis": multiple_sclerosis,
        "endometriosis": endometriosis, 
        "kidney disease": kidney_disease,
        "HIV/AIDS": HIV_AIDS 
    }

    changed_data = data.copy()
    changed_data['disease'] = data.disease.map(mapping_disease)
    return changed_data


def change_date_of_birth(data,zero, trente, quarante, cinquante, soixante , septante, quatre_vinght, nonante) :

    mapping_dob = {
        "0" : zero,
        "30" : trente,
        "40" : quarante,
        "50" : cinquante,
        "60" : soixante,
        "70" : septante,
        "80" : quatre_vinght,
        "90" : nonante
        
    }

    changed_data = data.copy()
    changed_data["dob"] = data.dob.map(mapping_dob)
    return changed_data


    


### tests

In [38]:
# 'low' under highschool, 'high' elsewhere 
X1 = change_education(data, less= '0', high='0', bach='1', master='1', phd='1')

# '0' male, '1' female 
X2 = change_gender(X1,male = 0, female = 1)

# active = {employed, student} unactive = {retired, unemployed}
X3 = change_employment(X2, employed= '1', retired='0', student='1', unemployed='0')

# QUE FAIRE AVEC LES CHILDREN ?
X4 = change_children(X3, "none", "one",'more than one','more than one','more than one','more than one','more than one','more than one','more than one')

# married = married, unmarried = others
X5 = change_marital_status(X4, divorced='0', married='1', single='0', widowed='1')

# native : north america, foreigners : others BUT to much 'other' compare to native so might affect negatively l-diversity. Instead, we merge north/south america 
X6 = change_ancestry(X5, Africa='Africa', Asia='Asia', Europe='Europe', North_America='America', Oceania='Oceania', South_America='America')

# 1 if got vehicule, 0 else 
X7 = change_number_vehicles(X6, zero='none', un='one', deux='more than one', trois='more than one')

# owner = owner, renter = renter regardless of the type of the accomodaations (2/3 owner, maybe reset these proportion ? )
X8 = change_accommodation(X7, Own_flat='1', Own_house='1', Rent_flat= '0', Rent_house='0', Rent_room='0')

# gathered under medical departments
X9 =change_disease(X8, gastritis='gastro-enterolgy', heart_disease='cardiovascular', hypertension='cardiovascular', diabetes='endocrynology', 
    skin_cancer='oncology', prostate_cancer='oncology', breast_cancer='oncology', 
    schizophrenia='psychiatry', Alzheimer='neurology', multiple_sclerosis='neurology', endometriosis='gynecology', kidney_disease='nephrology', HIV_AIDS='interal mdeicine')

# 

## Gestion ZIPCODE

import d'un base de données trouvée sur le net : http://structnet.com/instructions/zip_min_max_by_state.html

### HOSPITAL CASE

In [39]:
dict_states = {}

for (i,state) in enumerate(states['State Name']):
    dict_states[state] = [states['Zip Min'].iloc[i,], states['Zip Max'].iloc[i,]]     

In [40]:
def change_zipcode_state(df, dic_state):
    dict_zip_to_state = {}
    new_df = df.copy()

    for i,zip in enumerate(df['zipcode']):
        found = False
        for state, r in list(dict_states.items()):
            if zip in range(int(r[0]), int(r[1])):
                dict_zip_to_state[zip] = state
                found = True
        if not found:
            dict_zip_to_state[zip] = 'unknown state'

    new_df['zipcode'] = df.zipcode.map(dict_zip_to_state)
    return new_df

On créé le dictionnaire qui pour chaque clé : 'nom_etat', fait correspondre la valuer [zip_min, zip_max]

In [41]:
test_new_data_zip = change_zipcode_state(data,dict_states)


# K-Anonymity and L-Diversity 

## k-Anonymity

In [42]:
def check_k_anonymity(data, k, list_categorical_variable) : #k=2 --> 2 individus ne sont pas distingables via les quasi-identifiers
    """
    return: k-anonymity if take into account list_categorical_variable into the model
    """
    new_data = data.copy()

    group_by  = new_data[list_categorical_variable].groupby(list_categorical_variable)  # 
    rep = group_by[list_categorical_variable[0]].count() >= k
    n_k_personnes = len(rep)-sum(rep) # if k =2 142 faux --> 142 personnes définies de façon unique
                                      # nombre de groupes dans lesquels il y a k personnes ou moins 

    return n_k_personnes 

### Sociology

#### Changement

In [43]:
# 'low' under highschool, 'high' elsewhere 
edu = change_education(data_socio, less= '0', high='0', bach='0', master='1', phd='1')

gen = change_gender(edu,male = 0, female = 1)

employ = change_employment(gen, employed= '1', retired='0', student='1', unemployed='0')

child = change_children(employ, "none", "more than one",'more than one','more than one','more than one','more than one','more than one','more than one','more than one')

marital = change_marital_status(child, divorced='0', married='1', single='0', widowed='1')

ancestry = change_ancestry(marital, Africa='Autre', Asia='Autre', Europe='Occident', North_America='Occident', Oceania='Occident', South_America='Autre')

nmbrVec = change_number_vehicles(ancestry, zero='none', un='one', deux='one', trois='one')

accomo = change_accommodation(nmbrVec, Own_flat='0', Own_house='1', Rent_flat= '0', Rent_house='1', Rent_room='0')

disease = change_disease(accomo, gastritis='gastro-enterolgy', heart_disease='cardiovascular', hypertension='cardiovascular', diabetes='endocrynology', 
    skin_cancer='oncology', prostate_cancer='oncology', breast_cancer='oncology', 
    schizophrenia='psychiatry', Alzheimer='neurology', multiple_sclerosis='neurology', endometriosis='gynecology', kidney_disease='nephrology', HIV_AIDS='interal mdeicine')

year = change_date_of_birth(disease, zero=0, trente=2, quarante=2, cinquante=2, soixante=2, septante=1, quatre_vinght=1, nonante=0)

for i in range(len(year["dob"])):


    if(year.at[i,"commute_time"]>=0.1) :
        year.at[i,"commute_time"] = "1"
    elif(year.at[i,"commute_time"]<0.1) :
        year.at[i,"commute_time"] = "0"

final_socio = year




In [44]:
final_socio.head()

,id,gender,dob,zipcode,education,employment,children,marital_status,ancestry,number_vehicles,commute_time,accommodation,disease
0,c47ed1e2401af117d792899356875808fc468009e00545...,1,2,26904,0,0,more than one,1,Autre,one,0.0,1,neurology
1,9200e092cb89f3af990d24c9c6c4875f1b9a74c98657ba...,1,2,78088,1,0,more than one,1,Autre,one,0.0,0,neurology
2,457e8ab0e940da0028da57f4e5dd4e71f5c7bda98c413b...,1,1,51502,0,1,more than one,1,Autre,none,1.0,0,cardiovascular
3,b10eefa100ba13cbd732cdb52bbbe6f0714fbc389cfbf4...,1,2,54080,0,0,more than one,1,Autre,none,0.0,0,gynecology
4,531e1ab7da669ece5b348da0d87c376e7285b816962aa3...,1,2,68785,0,0,more than one,1,Occident,none,0.0,0,gastro-enterolgy


In [45]:
def check_k_anonymity(data, k, list_variable) :
    group_by  = data[list_variable].groupby(list_variable)
    is_k_anonym = group_by[list_variable[0]].count() >= k

    return is_k_anonym 

In [46]:
rep = check_k_anonymity(final_socio, 2, ["gender","dob", "education","employment",
                                         "children", "marital_status", "ancestry", "commute_time", "accommodation"])

print(rep)
print(len(rep)-sum(rep))


# SI ON PREND UN GROUPE SUPERIEUR A 2, L'INFORMATION DONNéE PAR LEN(REP) - SUM(REP) NE DONNERA PAS L'INFORMATION DU NOMBRE DE PERSONNE
# QUI NE FONT PAS PARTIE DU GROUPE ! !! ! !!!! ! !! ! !! ! IL FAUDRA AJOUTER UNE CONDITION, HEUREUSEMENT ON S'EN BRANLE COMPLèTEMENT 
# PUISQUE ON VEUT JUSTE DU 2 ANONYMITY ET DU 2 DIVERSITY !!!!!

gender  dob  education  employment  children       marital_status  ancestry  commute_time  accommodation
0       0    0          0           more than one  0               Autre     0.0           0                False
                                                                   Occident  0.0           1                False
                                                   1               Autre     0.0           0                 True
                                                                                           1                 True
                                                                   Occident  0.0           0                 True
                                                                                                            ...  
1       2    1          1           more than one  1               Occident  0.0           1                False
                                                                             1.0           0     

Pour sociology, on a un total de 176 valeur seule, ce qui représente environ 8,8% du dataframe.

#### 1ere fonction que je pense être fausse

In [ ]:
def check_L_diversity(data, k, list_categorical_variable, sensitive):

    group_by  = final_socio[list_categorical_variable+[sensitive]].groupby(list_categorical_variable)
    is_L_diverse = group_by[list_categorical_variable[0]].count()>=k
    return is_L_diverse


#### 2ème fonction qui je pense est bonne

In [146]:
ma_liste = ["gender","dob", "education","employment",
                                         "children", "marital_status", "ancestry", "commute_time", "accommodation"]

sensitive = "disease"

def check_L_diversity_mieux(data, k, list_categorical_variable, sensitive):
    group_by = data.groupby(list_categorical_variable)[sensitive]
    L_diverse = group_by.nunique()>=k

    return L_diverse


rep = check_L_diversity_mieux(final_socio,2, ma_liste, sensitive)
print(len(rep)-sum(rep))



97


#### TEST DIVERSITY

In [120]:
quasi_identifiers = ["gender","dob", "education","employment",
                                         "children", "marital_status", "ancestry", "commute_time", "accommodation"]
sensitive_column = "disease"


gbqi = final_socio[quasi_identifiers+[sensitive_column]].groupby(quasi_identifiers)

tamere = gbqi.nunique()>1
#print(len(tamere)-sum(tamere))
print(tamere)


                                                                                                  disease
gender dob education employment children      marital_status ancestry commute_time accommodation         
0      0   0         0          more than one 0              Autre    0.0          0                False
                                                             Occident 0.0          1                False
                                              1              Autre    0.0          0                 True
                                                                                   1                 True
                                                             Occident 0.0          0                 True
...                                                                                                   ...
1      2   1         1          more than one 1              Occident 0.0          1                False
                                              

### Hospital

In [ ]:
# 'low' under highschool, 'high' elsewhere 

child = change_children(data_hospi, "none", "more than one",'more than one','more than one','more than one','more than one','more than one','more than one','more than one')
nmbrVec = change_number_vehicles(child, zero='none', un='one', deux='one', trois='one')
disease = change_disease(nmbrVec, gastritis='gastro-enterolgy', heart_disease='cardiovascular', hypertension='cardiovascular', diabetes='endocrynology', 
    skin_cancer='oncology', prostate_cancer='oncology', breast_cancer='oncology', 
    schizophrenia='psychiatry', Alzheimer='neurology', multiple_sclerosis='neurology', endometriosis='gynecology', kidney_disease='nephrology', HIV_AIDS='interal mdeicine')

# date_of_birth
year = change_date_of_birth(disease, zero=0, trente=2, quarante=2, cinquante=2, soixante=2, septante=1, quatre_vinght=1, nonante=0)
zipuscodee = change_zipcode_state(year,dict_states)
final_hospi = zipuscodee

# date_of_birth


In [ ]:
rep = check_k_anonymity(final_hospi, 2, ["zipcode","children","dob", "number_vehicles"])


print(rep)
print(len(rep)-sum(rep))

zipcode        children       dob  number_vehicles
Alabama        more than one  0.0  none                True
                                   one                False
                              1.0  none                True
                                   one                 True
                              2.0  none                True
                                                      ...  
unknown state  none           0.0  one                 True
                              1.0  none               False
                                   one                 True
                              2.0  none               False
                                   one                 True
Name: zipcode, Length: 294, dtype: bool
84


In [ ]:
final_hospi

,id,gender,dob,zipcode,education,employment,children,marital_status,ancestry,number_vehicles,commute_time,accommodation,disease
0,e8799603521c0b2c53717c48fdb0138b0d388b2c1a85ab...,female,2.0,unknown state,High School,Retired,more than one,married,Africa,one,0.0,Own house,neurology
1,fb0becfbbca04c5205036b2ceaffb1a2bab789ca0248a0...,female,2.0,unknown state,PhD/md,Retired,more than one,married,Africa,one,0.0,Rent flat,neurology
2,7af7aa616d336fad0a6a8d44ba79b387be192048b1dc0e...,female,1.0,Iowa,Bachelor,Employed,more than one,married,Asia,none,0.1,Rent flat,cardiovascular
3,522f6731e80a3aa6b5f65c1d2d82366cae678ac862c86e...,female,2.0,Wisconsin,Bachelor,Retired,more than one,married,Africa,none,0.0,Rent room,gynecology
4,fea388953b1f5bf791c207cddfdb2ba179a02e7507f541...,female,2.0,Nebraska,High School,Retired,more than one,married,Europe,none,0.0,Rent flat,gastro-enterolgy
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,114ba144162b33e4317efa98ccb52705ef422189b42bf7...,male,NaN,Louisiana,High School,Student,none,divorced,Europe,none,1.1,Rent house,gastro-enterolgy
1996,d6826d1b204f29fe5f0ec74fdc27f2bfd35feee4478420...,male,NaN,Michigan,High School,Student,none,single,North America,none,0.1,Rent house,gastro-enterolgy
1997,67ebd049b81975994b9d30a6e10887a90caa1762054ad0...,female,2.0,california,Masters,Retired,more than one,married,Africa,one,0.0,Own house,oncology
1998,a132bf2adfeadb7102840af3f73e914d469f54cadb2165...,male,1.0,Kentucky,High School,Employed,more than one,married,Asia,one,0.0,Own flat,endocrynology
